# _utils

In [ ]:
#|default_exp _utils

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import repoyard._utils as this_module

In [ ]:
#|export
import subprocess
import shlex
import json
from enum import Enum
from repoyard import const
import repoyard.config
from pathlib import Path

Set up testing environment

In [ ]:
tests_working_dir = const.pkg_path.parent / "tmp_tests"
test_folder_path = tests_working_dir / "rclone_utils_test"
!rm -rf {test_folder_path}

In [ ]:
def setup_test_folder(rel_path):
    import shutil, inspect, os
    full_path = test_folder_path / rel_path
    shutil.rmtree(full_path, ignore_errors=True)
    full_path.mkdir(parents=True, exist_ok=True)
    
    (full_path / "my_local").mkdir(parents=True, exist_ok=True)
    (full_path / "my_local" / "file1.txt").write_text("Hello, world!")
    (full_path / "my_local" / "file2.txt").write_text("Goodbye, world!")
    (full_path / "my_remote").mkdir(parents=True, exist_ok=True)
    
    (full_path / "rclone.conf").write_text(inspect.cleandoc(f"""
    [my_remote]
    type = alias
    remote = {full_path / "my_remote"}
    """))
    
    return full_path

In [ ]:
#|hide
show_doc(this_module._rclone_cmd_helper)

## _rclone_cmd_helper

```python
_rclone_cmd_helper(
   cmd_name: str,
   rclone_config_path: str,
   source: str,
   source_path: str,
   dest: str,
   dest_path: str,
   exclude: list[str],
   dry_run: bool
) -> list[str]
```

---


In [ ]:
#|exporti
def _rclone_cmd_helper(
    cmd_name: str,
    rclone_config_path: str,
    source: str,
    source_path: str,   
    dest: str,
    dest_path: str,
    exclude: list[str],
    exclude_file: str|None,
    filters_file: str|None,
    dry_run: bool,
) -> list[str]:
    source_spec = f"{source}:{source_path}" if source else source_path
    dest_spec = f"{dest}:{dest_path}" if dest else dest_path
    cmd = ["rclone", cmd_name, '--config', rclone_config_path, source_spec, dest_spec]
    if dry_run:
        cmd.append("--dry-run")
    for f in exclude:
        cmd.append(f"--exclude")
        cmd.append(f)
    if exclude_file is not None:
        cmd.append(f"--exclude-from")
        cmd.append(exclude_file)
    if filters_file is not None:
        cmd.append("--filters-file")
        cmd.append(filters_file)
    return cmd

In [ ]:
#|hide
show_doc(this_module._remove_ansi_escape)

## _remove_ansi_escape

```python
_remove_ansi_escape(text: str) -> str
```

---


In [ ]:
#|exporti
# Source - https://stackoverflow.com/a
# Posted by Martijn Pieters, modified by community. See post 'Timeline' for change history
# Retrieved 2025-11-10, License - CC BY-SA 4.0

import re
ansi_escape = re.compile(r'''
    \x1B  # ESC
    (?:   # 7-bit C1 Fe (except CSI)
        [@-Z\\-_]
    |     # or [ for CSI, followed by a control sequence
        \[
        [0-?]*  # Parameter bytes
        [ -/]*  # Intermediate bytes
        [@-~]   # Final byte
    )
''', re.VERBOSE)

def _remove_ansi_escape(text: str) -> str:
    return ansi_escape.sub('', text)

In [ ]:
_remove_ansi_escape("Hello \x1B[31mWorld\x1B[0m")

'Hello World'

In [ ]:
#|hide
show_doc(this_module.rclone_copy)

## rclone_copy

```python
rclone_copy(
   rclone_config_path: str,
   source: str,
   source_path: str,
   dest: str,
   dest_path: str,
   exclude: list[str],
   dry_run: bool,
   return_command: bool,
   verbose
) -> bool
```

---


In [ ]:
#|export
def rclone_copy(
    rclone_config_path: str,
    source: str,
    source_path: str,   
    dest: str,
    dest_path: str,
    exclude: list[str],
    exclude_file: str|None,
    filters_file: str|None,
    dry_run: bool,
    return_command: bool=False,
    verbose=True,
) -> bool:
    cmd = _rclone_cmd_helper("copy", rclone_config_path, source, source_path, dest, dest_path, exclude, exclude_file, filters_file, dry_run)
    if not return_command:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if verbose:
            print(result.stdout)
            print(result.stderr)
        return result.returncode == 0, result.stdout, result.stderr
    else:
        return shlex.join(cmd)

In [ ]:
_path = setup_test_folder('copy')


res = rclone_copy(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_local",
    dest="my_remote",
    dest_path="",
    exclude=[],
    dry_run=False,
    verbose=True,
)

assert res
ls = [f.name for f in (_path / "my_remote").ls()]
assert "file1.txt" in ls
assert "file2.txt" in ls

In [ ]:
#|hide
show_doc(this_module.rclone_sync)

## rclone_sync

```python
rclone_sync(
   rclone_config_path: str,
   source: str,
   source_path: str,
   dest: str,
   dest_path: str,
   exclude: list[str],
   dry_run: bool,
   return_command: bool,
   verbose
) -> bool
```

---


In [ ]:
#|export
def rclone_sync(
    rclone_config_path: str,
    source: str,
    source_path: str,   
    dest: str,
    dest_path: str,
    exclude: list[str],
    exclude_file: str|None,
    filters_file: str|None,
    dry_run: bool,
    return_command: bool=False,
    verbose=True,
) -> bool:
    cmd = _rclone_cmd_helper("sync", rclone_config_path, source, source_path, dest, dest_path, exclude, exclude_file, filters_file, dry_run)
    if not return_command:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if verbose:
            print(result.stdout)
            print(result.stderr)
        return result.returncode == 0, result.stdout, result.stderr
    else:
        return shlex.join(cmd)

In [ ]:
_path = setup_test_folder('sync')

res = rclone_sync(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_local",
    dest="my_remote",
    dest_path="",
    exclude=[],
    dry_run=False,
    verbose=True,
)

assert res
ls = [f.name for f in (_path / "my_remote").ls()]
assert "file1.txt" in ls
assert "file2.txt" in ls

In [ ]:
#|hide
show_doc(this_module.rclone_bisync)

## rclone_bisync

```python
rclone_bisync(
   rclone_config_path: str,
   source: str,
   source_path: str,
   dest: str,
   dest_path: str,
   exclude: list[str],
   dry_run: bool,
   resync: bool,
   force: bool,
   return_command: bool,
   verbose: bool
) -> BisyncResult
```

---


In [ ]:
#|export
class BisyncResult(Enum):
    SUCCESS = "success"
    CONFLICTS = "conflicts"
    ERROR_NEEDS_RESYNC = "needs_resync"
    ERROR_ALL_FILES_CHANGED = "all_files_changed"
    ERROR_OTHER = "other_error"

def rclone_bisync(
    rclone_config_path: str,
    source: str,
    source_path: str,   
    dest: str,
    dest_path: str,
    exclude: list[str],
    exclude_file: str|None,
    filters_file: str|None,
    dry_run: bool,
    resync: bool,
    force: bool,
    return_command: bool=False,
    verbose: bool=False,
) -> BisyncResult:
    cmd = _rclone_cmd_helper("bisync", rclone_config_path, source, source_path, dest, dest_path, exclude, exclude_file, filters_file, dry_run)
    if resync: cmd.append("--resync")
    if force: cmd.append("--force")
        
    if not return_command:
        result = subprocess.run(cmd, capture_output=True, text=True)
        if verbose:
            print(result.stdout)
            print(result.stderr)
        stdout_clean = _remove_ansi_escape(result.stdout)
        stderr_clean = _remove_ansi_escape(result.stderr)
        if "ERROR : Bisync aborted. Must run --resync to recover." in stderr_clean:
            return BisyncResult.ERROR_NEEDS_RESYNC, result.stdout, result.stderr
        if "ERROR : Safety abort: all files were changed" in stderr_clean:
            return BisyncResult.ERROR_ALL_FILES_CHANGED, result.stdout, result.stderr
        if result.returncode != 0:
            return BisyncResult.ERROR_OTHER, result.stdout, result.stderr
        if "NOTICE: - WARNING  New or changed in both paths" in stderr_clean:
            return BisyncResult.CONFLICTS, result.stdout, result.stderr
        return BisyncResult.SUCCESS, result.stdout, result.stderr
    else:
        return shlex.join([c.as_posix() if type(c) == Path else str(c) for c in cmd])

In [ ]:
_path = setup_test_folder('bisync')

res = rclone_bisync(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_local",
    dest="my_remote",
    dest_path="",
    exclude=[],
    dry_run=False,
    resync=False,
    force=False,
    verbose=True,
)

assert res == BisyncResult.ERROR_NEEDS_RESYNC


2025/11/11 18:29:42 ERROR : Empty current Path2 listing. Cannot sync to an empty directory: /Users/lukastk/Library/Caches/rclone/bisync/Users_lukastk_dev_2025-11-09_00__repoyard_tmp_tests_rclone_utils_test_bisync_my_local..Users_lukastk_dev_2025-11-09_00__repoyard_tmp_tests_rclone_utils_test_bisync_my_remote.path2.lst-new
2025/11/11 18:29:42 ERROR : Bisync critical error: empty current Path2 listing: /Users/lukastk/Library/Caches/rclone/bisync/Users_lukastk_dev_2025-11-09_00__repoyard_tmp_tests_rclone_utils_test_bisync_my_local..Users_lukastk_dev_2025-11-09_00__repoyard_tmp_tests_rclone_utils_test_bisync_my_remote.path2.lst-new
2025/11/11 18:29:42 ERROR : Bisync aborted. Must run --resync to recover.
2025/11/11 18:29:42 NOTICE: Failed to bisync: bisync aborted



In [ ]:
_path = setup_test_folder('bisync')

res = rclone_bisync(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_local",
    dest="my_remote",
    dest_path="",
    exclude=[],
    dry_run=False,
    resync=True,
    force=False,
    verbose=True,
)

assert res == BisyncResult.SUCCESS

In [ ]:
!echo "will cause conflict" > {_path / "my_local" / "file1.txt"}
!echo "will cause conflict!" > {_path / "my_remote" / "file1.txt"}

res = rclone_bisync(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_local",
    dest="my_remote",
    dest_path="",
    exclude=[],
    dry_run=False,
    resync=False,
    force=False,
    verbose=True,
)

assert res == BisyncResult.CONFLICTS


2025/11/11 18:29:42 NOTICE: - WARNING  New or changed in both paths       - file1.txt
2025/11/11 18:29:42 NOTICE: - Path1    Renaming Path1 copy                - /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/rclone_utils_test/bisync/my_local/file1.txt.conflict1
2025/11/11 18:29:42 NOTICE: - Path1    Queue copy to Path2       - /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/rclone_utils_test/bisync/my_remote/file1.txt.conflict1
2025/11/11 18:29:42 NOTICE: - Path2    Renaming Path2 copy                - /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/rclone_utils_test/bisync/my_remote/file1.txt.conflict2
2025/11/11 18:29:42 NOTICE: - Path2    Queue copy to Path1       - /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/rclone_utils_test/bisync/my_local/file1.txt.conflict2



In [ ]:
#|hide
show_doc(this_module.rclone_mkdir)

In [ ]:
#|export
def rclone_mkdir(
    rclone_config_path: str,
    source: str,
    source_path: str,
) -> dict|None:
    source_str = f"{source}:{source_path}" if source else source_path
    cmd = ["rclone", "mkdir", '--config', rclone_config_path, source_str]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise Exception(result.stderr)

In [ ]:
#|hide
show_doc(this_module.rclone_lsjson)

## rclone_lsjson

```python
rclone_lsjson(
   rclone_config_path: str,
   source: str,
   source_path: str
) -> dict | None
```

---


In [ ]:
#|export
def rclone_lsjson(
    rclone_config_path: str,
    source: str,
    source_path: str,
) -> dict|None:
    source_str = f"{source}:{source_path}" if source else source_path
    cmd = ["rclone", "lsjson", '--config', rclone_config_path, source_str]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        return None
    return json.loads(result.stdout)

In [ ]:
_path = setup_test_folder('lsjson')

res = rclone_bisync(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_local",
    dest="my_remote",
    dest_path="",
    exclude=[],
    dry_run=False,
    resync=True,
    force=False,
    verbose=True,
)

res = rclone_lsjson(
    _path / "rclone.conf",
    source="my_remote",
    source_path="",
)
file_names = [f["Name"] for f in res]
assert "file1.txt" in file_names
assert "file2.txt" in file_names

In [ ]:
rclone_lsjson(
    _path / "rclone.conf",
    source="my_remote",
    source_path="",
)

[{'Path': 'file1.txt',
  'Name': 'file1.txt',
  'Size': 13,
  'MimeType': 'text/plain; charset=utf-8',
  'ModTime': '2025-11-11T18:29:42.821689746Z',
  'IsDir': False},
 {'Path': 'file2.txt',
  'Name': 'file2.txt',
  'Size': 15,
  'MimeType': 'text/plain; charset=utf-8',
  'ModTime': '2025-11-11T18:29:42.821859247Z',
  'IsDir': False}]

In [ ]:
#|hide
show_doc(this_module.rclone_path_exists)

## rclone_path_exists

```python
rclone_path_exists(
   rclone_config_path: str,
   source: str,
   source_path: str
) -> tuple[bool, bool]
```

Check if a path exists in rclone.

Returns a tuple of (exists, is_dir).

---


In [ ]:
#|export
def rclone_path_exists(
    rclone_config_path: str,
    source: str,
    source_path: str,
) -> tuple[bool, bool]:
    """
    Check if a path exists in rclone.
    Returns a tuple of (exists, is_dir).
    """
    parent_path = Path(source_path).parent if len(Path(source_path).parts) > 1 else ""
    ls = rclone_lsjson(
        rclone_config_path,
        source,
        parent_path,
    )
    if ls is None:
        return (False, False)
    ls = {f["Name"]: f for f in ls}
    exists = Path(source_path).name in ls
    is_dir = ls[Path(source_path).name]["IsDir"]
    return (exists, is_dir)

In [ ]:
assert rclone_path_exists(
    _path / "rclone.conf",
    source="my_remote",
    source_path="file1.txt",
) == (True, False)

In [ ]:
assert rclone_path_exists(
    _path / "rclone.conf",
    source="",
    source_path=_path / "my_remote",
) == (True, True)

In [ ]:
#|hide
show_doc(this_module.get_synced_repo_full_name_from_sub_path)

## get_synced_repo_full_name_from_sub_path

```python
get_synced_repo_full_name_from_sub_path(
   config: Config,
   sub_path: str
) -> pathlib.Path | None
```

Get the full name of a synced repo from a path inside of the repo.

---


In [ ]:
#|export
def get_synced_repo_full_name_from_sub_path(
    config: repoyard.config.Config,
    sub_path: str,
) -> Path|None:
    """
    Get the full name of a synced repo from a path inside of the repo.
    """
    sub_path = Path(sub_path).expanduser()
    is_in_synced_repostore = sub_path.is_relative_to(config.included_repostore_path)
    
    if not is_in_synced_repostore:
        return None
    
    repo_full_name = sub_path.relative_to(config.included_repostore_path).parts[0]
    return repo_full_name

In [ ]:
#|hide
show_doc(this_module.get_hostname)

## get_hostname

```python
get_hostname()
```

---


In [ ]:
#|export
import platform
import subprocess

def get_hostname():
    system = platform.system()
    hostname = None
    if system == "Darwin":
        # Mac
        try:
            result = subprocess.run(["scutil", "--get", "ComputerName"], capture_output=True, text=True, check=True)
            hostname = result.stdout.strip()
        except Exception:
            hostname = None
    if hostname is None:
        hostname = platform.node()
    return hostname